In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
url = 'https://docs.google.com/spreadsheets/d/1_y0_LJmSY6sNx8qd51T70n0oa_ugN50AVFKuJmXO1-s/export?format=csv&gid=2042605142'
approval = pd.read_csv(url)
approval

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,url_topline,url_crosstab,race_id,cycle,office_type,seat_name,election_date,stage,dem,rep
0,89437,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2024/12...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,48.4
1,89445,294,McLaughlin,NaN,NaN,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,NaN,https://mclaughlinonline.com/pols/wp-content/u...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,42.0,47.0
2,89491,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2025/01...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,46.8
3,89497,1554,RMG Research,2178,Napolitan Institute,NaN,NaN,NaN,RMG Research,Online Opt-In Panel,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.0,51.0
4,89511,1895,Quantus Insights,2184,TrendingPolitics,NaN,NaN,NaN,Quantus Insights,Online Opt-In Panel,...,NaN,https://drive.google.com/file/d/1d1e9IDGzoiE2S...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,45.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,90709,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,https://d3nkl3psvxxpe9.cloudfront.net/document...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.0,41.0
140,90710,1554,RMG Research,2178,Napolitan Institute,NaN,NaN,NaN,RMG Research,Online Opt-In Panel,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,45.0,46.0
141,90714,1952,Verasight/The Argument,NaN,NaN,NaN,NaN,NaN,The Argument/Verasight,Text/Online Ad/Email,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,53.0,47.0
142,90716,568,YouGov,1213,Yahoo News,NaN,NaN,NaN,YouGov,Online Matched Sample,...,NaN,https://www.scribd.com/document/925455404/Comb...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.0,40.0


In [4]:
approval = approval[~approval['poll_id'].isna()]
approval

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,url_topline,url_crosstab,race_id,cycle,office_type,seat_name,election_date,stage,dem,rep
0,89437,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2024/12...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,48.4
1,89445,294,McLaughlin,NaN,NaN,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,NaN,https://mclaughlinonline.com/pols/wp-content/u...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,42.0,47.0
2,89491,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2025/01...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,46.8
3,89497,1554,RMG Research,2178,Napolitan Institute,NaN,NaN,NaN,RMG Research,Online Opt-In Panel,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.0,51.0
4,89511,1895,Quantus Insights,2184,TrendingPolitics,NaN,NaN,NaN,Quantus Insights,Online Opt-In Panel,...,NaN,https://drive.google.com/file/d/1d1e9IDGzoiE2S...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,45.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,90709,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,https://d3nkl3psvxxpe9.cloudfront.net/document...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.0,41.0
140,90710,1554,RMG Research,2178,Napolitan Institute,NaN,NaN,NaN,RMG Research,Online Opt-In Panel,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,45.0,46.0
141,90714,1952,Verasight/The Argument,NaN,NaN,NaN,NaN,NaN,The Argument/Verasight,Text/Online Ad/Email,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,53.0,47.0
142,90716,568,YouGov,1213,Yahoo News,NaN,NaN,NaN,YouGov,Online Matched Sample,...,NaN,https://www.scribd.com/document/925455404/Comb...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.0,40.0


In [5]:
approval.columns.values

array(['poll_id', 'pollster_id', 'pollster', 'sponsor_ids', 'sponsors',
       'sponsor_candidate_id', 'sponsor_candidate',
       'sponsor_candidate_party', 'display_name', 'mode', 'state',
       'start_date', 'end_date', 'internal', 'partisan', 'question_id',
       'sample_size', 'population', 'subpopulation', 'population_full',
       'tracking', 'created_at', 'updated_at', 'notes', 'url',
       'url_article', 'url_topline', 'url_crosstab', 'race_id', 'cycle',
       'office_type', 'seat_name', 'election_date', 'stage', 'dem', 'rep'],
      dtype=object)

In [6]:
# morris_to_silver_dict = {
#     'HarrisX/Harris Poll':'Harris Insights & Analytics',
#     'Clarity': 'Clarity Campaign Labs',
#     'Beacon/Shaw': 'Beacon Research/Shaw & Co. Research',
#     'Monmouth': 'Monmouth University',
#     'HarrisX': 'Harris Insights & Analytics',
#     'UMass Amherst/YouGov': 'YouGov',
#     'Emerson': 'Emerson College',
#     'Marquette Law School': 'Marquette University Law School',
#     'NewsNation/Decision Desk HQ': 'NewsNation',
#     'CNN/SSRS': 'SSRS',
#     'Suffolk': 'Suffolk University',
#     'Harris Poll': 'Harris Insights & Analytics',
#     'Fabrizio/Impact': 'Fabrizio Lee & Associates/Impact Research',
#     'Winston': 'The Winston Group',
#     'Hart/POS': 'Hart Research Associates/Public Opinion Strategies',
#     'Quinnipiac': 'Quinnipiac University',
#     'Siena/NYT': 'The New York Times/Siena College',
#     'Noble Predictive Insights': 'OH Predictive Insights',
#     'McLaughlin': 'McLaughlin & Associates'
# }

In [7]:
pollster_key = pd.DataFrame(approval[['pollster', 'display_name']].value_counts()).reset_index()
pollster_key.head()

,pollster,display_name,count
0,YouGov,YouGov,36
1,McLaughlin,McLaughlin & Associates,19
2,Cygnal,Cygnal,11
3,Echelon Insights,Echelon Insights,10
4,Quantus Insights,Quantus Insights,9


In [8]:
pollster_key

,pollster,display_name,count
0,YouGov,YouGov,36
1,McLaughlin,McLaughlin & Associates,19
2,Cygnal,Cygnal,11
3,Echelon Insights,Echelon Insights,10
4,Quantus Insights,Quantus Insights,9
5,RMG Research,RMG Research,9
6,Clarity,Clarity Campaign Labs,6
7,Verasight,Verasight,5
8,Emerson,Emerson College,4
9,Hart/POS,Hart Research Associates/Public Opinion Strate...,3


In [9]:
# pollsters_short = np.unique(approval['pollster'])
# pollsters_long = np.unique(approval['display_name'])

In [10]:
# pollsters_short, pollsters_long

In [11]:
# pollsters_short.shape, pollsters_long.shape

In [12]:
morris_to_silver_dict = {short: long for short, long in zip(approval['pollster'], approval['display_name'])}
morris_to_silver_dict

{'Cygnal': 'Cygnal',
 'McLaughlin': 'McLaughlin & Associates',
 'RMG Research': 'RMG Research',
 'Quantus Insights': 'Quantus Insights',
 'Fabrizio/Impact': 'Fabrizio, Lee & Associates/Impact Research',
 'Clarity': 'Clarity Campaign Labs',
 'Echelon Insights': 'Echelon Insights',
 'co/efficient': 'co/efficient',
 'YouGov': 'YouGov',
 'Public Opinion Strategies': 'Public Opinion Strategies',
 'Emerson': 'Emerson College',
 'Hart/POS': 'Hart Research Associates/Public Opinion Strategies',
 'Normington, Petts & Associates': 'Normington, Petts & Associates',
 'Yale Youth Poll': 'Yale University Institution for Social and Policy Studies',
 'Winston': 'The Winston Group',
 'RealClear Opinion Research': 'RealClear Opinion Research',
 'Noble Predictive Insights': 'Noble Predictive Insights',
 'Beacon/Shaw': 'Beacon Research/Shaw & Co. Research',
 'Siena/NYT': 'The New York Times/Siena College',
 'NewsNation/Decision Desk HQ': 'NewsNation',
 'Big Data Poll': 'Big Data Poll',
 'Verasight': 'Vera

In [13]:
morris_to_silver_dict.update({
    'HarrisX/Harris Poll':'Harris Insights & Analytics',
    'Clarity': 'Clarity Campaign Labs',
    'Beacon/Shaw': 'Beacon Research/Shaw & Co. Research',
    'Monmouth': 'Monmouth University',
    'HarrisX': 'Harris Insights & Analytics',
    'UMass Amherst/YouGov': 'YouGov',
    'Emerson': 'Emerson College',
    'Marquette Law School': 'Marquette University Law School',
    'NewsNation/Decision Desk HQ': 'NewsNation',
    'CNN/SSRS': 'SSRS',
    'Suffolk': 'Suffolk University',
    'Harris Poll': 'Harris Insights & Analytics',
    'Fabrizio/Impact': 'Fabrizio Lee & Associates/Impact Research',
    'Winston': 'Verasight',
    'Verasight': 'Verasight',
    'Hart/POS': 'Hart Research Associates/Public Opinion Strategies',
    'Quinnipiac': 'Quinnipiac University',
    'Siena/NYT': 'The New York Times/Siena College',
    'Noble Predictive Insights': 'OH Predictive Insights',
    'McLaughlin': 'McLaughlin & Associates',
    'Verasight/The Argument': 'Verasight'
})

In [14]:
morris_to_silver_dict

{'Cygnal': 'Cygnal',
 'McLaughlin': 'McLaughlin & Associates',
 'RMG Research': 'RMG Research',
 'Quantus Insights': 'Quantus Insights',
 'Fabrizio/Impact': 'Fabrizio Lee & Associates/Impact Research',
 'Clarity': 'Clarity Campaign Labs',
 'Echelon Insights': 'Echelon Insights',
 'co/efficient': 'co/efficient',
 'YouGov': 'YouGov',
 'Public Opinion Strategies': 'Public Opinion Strategies',
 'Emerson': 'Emerson College',
 'Hart/POS': 'Hart Research Associates/Public Opinion Strategies',
 'Normington, Petts & Associates': 'Normington, Petts & Associates',
 'Yale Youth Poll': 'Yale University Institution for Social and Policy Studies',
 'Winston': 'Verasight',
 'RealClear Opinion Research': 'RealClear Opinion Research',
 'Noble Predictive Insights': 'OH Predictive Insights',
 'Beacon/Shaw': 'Beacon Research/Shaw & Co. Research',
 'Siena/NYT': 'The New York Times/Siena College',
 'NewsNation/Decision Desk HQ': 'NewsNation',
 'Big Data Poll': 'Big Data Poll',
 'Verasight': 'Verasight',
 'Fa

In [15]:
approval['pollster'] = approval['pollster'].replace(morris_to_silver_dict)
approval = approval.rename({'population':'population_empty'}, axis='columns')
approval = approval.rename({'population_full':'population'}, axis='columns')
approval.head(7)

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,url_topline,url_crosstab,race_id,cycle,office_type,seat_name,election_date,stage,dem,rep
0,89437,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2024/12...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,48.4
1,89445,294,McLaughlin & Associates,NaN,NaN,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,NaN,https://mclaughlinonline.com/pols/wp-content/u...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,42.0,47.0
2,89491,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2025/01...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,46.8
3,89497,1554,RMG Research,2178,Napolitan Institute,NaN,NaN,NaN,RMG Research,Online Opt-In Panel,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.0,51.0
4,89511,1895,Quantus Insights,2184,TrendingPolitics,NaN,NaN,NaN,Quantus Insights,Online Opt-In Panel,...,NaN,https://drive.google.com/file/d/1d1e9IDGzoiE2S...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,45.0,48.0
5,89530,294,McLaughlin & Associates,NaN,NaN,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,NaN,https://mclaughlinonline.com/pols/wp-content/u...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,40.0,48.0
6,89555,1699,Fabrizio Lee & Associates/Impact Research,42,AARP,NaN,NaN,NaN,"Fabrizio, Lee & Associates/Impact Research",Online Opt-In Panel/Text-to-Web,...,NaN,https://www.aarp.org/content/dam/aarp/research...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,43.0,43.0


In [16]:
approval.shape

(144, 36)

In [17]:
approval = approval[approval['partisan'].isna()]# [pd.to_datetime(approval['end_date']) >= pd.to_datetime('2023-12-01')]
approval.shape

(122, 36)

In [18]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [19]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [20]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(120, 36)

In [21]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='left')
approval.shape

(120, 38)

In [22]:
approval = approval[(~approval['sample_size'].isna())]
approval.shape

(119, 38)

In [23]:
approval['pollster'].value_counts()

pollster
YouGov                                                       36
Cygnal                                                       11
Echelon Insights                                             10
McLaughlin & Associates                                       9
RMG Research                                                  9
Verasight                                                     7
Clarity Campaign Labs                                         6
Emerson College                                               4
Fabrizio Lee & Associates/Impact Research                     3
Quantus Insights                                              3
AtlasIntel                                                    3
The New York Times/Siena College                              2
SSRS                                                          2
Hart Research Associates/Public Opinion Strategies            2
RealClear Opinion Research                                    1
Beacon Research/Shaw & Co. Rese

In [24]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [25]:
# house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+\d.\d\d.*@@([\d\.\+\-]+)')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,-1.999072
1,D,1.017374
2,D,0.632896
3,D,0.791334
4,R,-0.597041
...,...,...
93,R,-2.443137
94,R,-0.446172
95,R,-3.234717
96,R,-4.939213


In [26]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[1]
house_eff

,0,1,house_effect
0,-1,-1.999072,-1.999072
1,1,1.017374,1.017374
2,1,0.632896,0.632896
3,1,0.791334,0.791334
4,-1,-0.597041,-0.597041
...,...,...,...
93,-1,-2.443137,-2.443137
94,-1,-0.446172,-0.446172
95,-1,-3.234717,-3.234717
96,-1,-4.939213,-4.939213


In [27]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-1.999072
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.017374
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.632896
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.791334
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.597041
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.443137
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.446172
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.234717


In [28]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['dem_adj'] = df['dem'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['rep_adj'] = df['rep'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['dem_adj'] = df['dem']
        df['rep_adj'] = df['rep']
    return df

In [29]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,office_type,seat_name,election_date,stage,dem,rep,Pollster,Predictive Plus-Minus,dem_adj,rep_adj
9,89579,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,35.0,37.0,YouGov,0.047635,34.750413,37.249587
10,89579,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,42.0,41.0,YouGov,0.047635,41.750413,41.249587
23,89690,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,39.0,36.0,YouGov,0.047635,38.750413,36.249587
24,89690,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.750413,42.249587
38,89897,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.750413,42.249587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,89658,379,Public Opinion Strategies,2255,Paragon Health Institute,NaN,NaN,NaN,Public Opinion Strategies,Live Phone,...,U.S. House,Generic Ballot,2026-11-03,general,44.0,46.0,Public Opinion Strategies,-0.246974,46.071409,43.928591
60,90198,1810,American Pulse Research & Polling,NaN,NaN,NaN,NaN,NaN,American Pulse Research & Polling,Live Phone/Online Opt-In Panel/Text-to-Web,...,U.S. House,Generic Ballot,2026-11-03,general,46.6,42.4,American Pulse Research & Polling,0.800463,46.600000,42.400000
91,90508,241,Ipsos,71,Reuters,NaN,NaN,NaN,Ipsos,Probability Panel,...,U.S. House,Generic Ballot,2026-11-03,general,38.0,34.0,Ipsos,0.442898,37.612631,34.387369
96,90535,1189,Morning Consult,NaN,NaN,NaN,NaN,NaN,Morning Consult,Online Opt-In Panel,...,U.S. House,Generic Ballot,2026-11-03,general,45.0,41.0,Morning Consult,0.584994,44.570341,41.429659


In [30]:
approval = approval_adj.copy()
approval.shape

(119, 40)

In [31]:
approval['pollster_id'] = pd.factorize(approval['pollster'] + approval['sponsors'].fillna('none'))[0]
approval.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,office_type,seat_name,election_date,stage,dem,rep,Pollster,Predictive Plus-Minus,dem_adj,rep_adj
9,89579,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,35.0,37.0,YouGov,0.047635,34.750413,37.249587
10,89579,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,42.0,41.0,YouGov,0.047635,41.750413,41.249587
23,89690,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,39.0,36.0,YouGov,0.047635,38.750413,36.249587
24,89690,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.750413,42.249587
38,89897,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.750413,42.249587


In [32]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id')]
    ndup = data[~data.duplicated(subset='poll_id')]
    #try:
    dup_a = dup[dup['population'].isin(['lv'])]
#     except ValueError:
#         print(dup['population'].isin(['a']))
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id')]
    ndup = df[~df.duplicated(subset='poll_id')]
    dup_lv = dup[dup['population'].isin(['rv'])]
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['start_date'] = pd.to_datetime(df['start_date'])
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [33]:
approval = approval.rename({'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [34]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'TIPP Insights', 'Trafalgar Group',
       'Trafalgar Group/Insider Advantage'], dtype=object)

In [35]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,office_type,seat_name,election_date,stage,dem,rep,Pollster,pred_plus_minus,dem_adj,rep_adj
9,89579,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,35.0,37.0,YouGov,0.047635,34.750413,37.249587
10,89579,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,42.0,41.0,YouGov,0.047635,41.750413,41.249587
23,89690,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,39.0,36.0,YouGov,0.047635,38.750413,36.249587
24,89690,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.750413,42.249587
38,89897,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.750413,42.249587


In [36]:
approval.shape

(119, 40)

In [37]:
# net: negative --> dem advantage, positive --> rep advantage
approval['net'] = approval['rep'] - approval['dem']
approval.head(1)

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,seat_name,election_date,stage,dem,rep,Pollster,pred_plus_minus,dem_adj,rep_adj,net
9,89579,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,Generic Ballot,2026-11-03,general,35.0,37.0,YouGov,0.047635,34.750413,37.249587,2.0


In [38]:
approval['url'] = approval['url_crosstab'].fillna(approval['url_topline']).fillna(approval['url_article'].fillna(''))

In [39]:
approval_polls = approval[['pollster', 'start_date', 'end_date', 'sample_size', 'population', 'dem', 'rep', 'dem_adj', 'rep_adj', 'net', 'pred_plus_minus', 'poll_id', 'url', 'sponsors', 'pollster_id']]
approval_polls

,pollster,start_date,end_date,sample_size,population,dem,rep,dem_adj,rep_adj,net,pred_plus_minus,poll_id,url,sponsors,pollster_id
9,YouGov,2025-02-16,2025-02-18,1603.0,a,35.0,37.0,34.750413,37.249587,2.0,0.047635,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
10,YouGov,2025-02-16,2025-02-18,1451.0,rv,42.0,41.0,41.750413,41.249587,-1.0,0.047635,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
23,YouGov,2025-04-05,2025-04-08,1741.0,a,39.0,36.0,38.750413,36.249587,-3.0,0.047635,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
24,YouGov,2025-04-05,2025-04-08,1558.0,rv,43.0,42.0,42.750413,42.249587,-1.0,0.047635,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
38,YouGov,2025-05-16,2025-05-19,1558.0,rv,43.0,42.0,42.750413,42.249587,-1.0,0.047635,89897,https://today.yougov.com/topics/politics/track...,The Economist,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,Public Opinion Strategies,2025-02-25,2025-03-02,1000.0,rv,44.0,46.0,46.071409,43.928591,2.0,-0.246974,89658,https://paragoninstitute.org/medicaid/paragon-...,Paragon Health Institute,26
60,American Pulse Research & Polling,2025-06-23,2025-06-25,633.0,rv,46.6,42.4,46.600000,42.400000,-4.2,0.800463,90198,https://drive.google.com/file/d/1fQWKf1QhBKMmx...,NaN,27
91,Ipsos,2025-08-22,2025-08-24,1022.0,a,38.0,34.0,37.612631,34.387369,-4.0,0.442898,90508,https://www.ipsos.com/sites/default/files/Reut...,Reuters,28
96,Morning Consult,2025-08-29,2025-08-31,2202.0,rv,45.0,41.0,44.570341,41.429659,-4.0,0.584994,90535,https://links.morningconsult.com/s/vb/zpOJTp1f...,NaN,29


In [40]:
def zone_flood_checker(data, pid, date):
    """
    Check whether there are multiple polls from the same pollster + same sponsor (if applicable) within the last two weeks, 
    which could indicate "flooding the zone." Return number of polls from same pollster/sponsor in last two weeks.

    :param data: Polls dataframe.
    :param pid: Pollster ID, specified in the polls dataframe as the column pollster_id.
    :param date: Date at which specified poll is conducted. Corresponds to column end_date in polls dataframe.
    """
    thres = date - pd.Timedelta(weeks=2)
    df = data[ (data['pollster_id'] == pid) & (pd.to_datetime(data['end_date']) >= thres) ]
    return max(df.shape[0], 1)

In [41]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    # state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    # linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    linear_weights = delta.map(lambda x: (1 - x/60) if x < 60 else 0)
    exp_weights = 0.1**(delta/(45))
    state_pivot['time_weights'] =  0.1 * linear_weights + 0.9 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if np.isnan(rel_qual):
            return 0.2
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'rv':
            return 0.8
        elif population == 'a':
            return 0.5
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

    # Multiple polls in short window (aka "flooding the zone")
    def zone_flood_weight(pid, date):
        num_polls = zone_flood_checker(approval_polls, pid, date)
        return 1 / np.sqrt(num_polls)
    state_pivot['zone_flood_weight'] = state_pivot[['pollster_id', 'end_date']].apply(
        lambda x: zone_flood_weight(x['pollster_id'], x['end_date']),
        axis=1
    )

    # Gather the weights together
    state_pivot['total_weights'] = (state_pivot['sample_size_weights'] * state_pivot['time_weights'] * 
                                    state_pivot['quality_weights'] * state_pivot['population_weights'] * state_pivot['zone_flood_weight'])
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [42]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,start_date,end_date,sample_size,population,dem,rep,dem_adj,rep_adj,net,...,poll_id,url,sponsors,pollster_id,sample_size_weights,time_weights,quality_weights,population_weights,zone_flood_weight,total_weights
68,Echelon Insights,2025-07-10,2025-07-14,1084.0,lv,45.0,41.0,45.000000,41.000000,-4.0,...,90309,https://echelonin.wpenginepowered.com/wp-conte...,NaN,3,0.008607,0.015801,1.181365,1.0,0.500000,1.378139e-03
9,YouGov,2025-02-16,2025-02-18,1603.0,a,35.0,37.0,34.750413,37.249587,2.0,...,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0,0.010467,0.000009,0.964021,0.5,0.174078,1.355957e-07
23,YouGov,2025-04-05,2025-04-08,1741.0,a,39.0,36.0,38.750413,36.249587,-3.0,...,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0,0.010908,0.000110,0.964021,0.5,0.179605,1.789137e-06
38,YouGov,2025-05-16,2025-05-19,1558.0,rv,43.0,42.0,42.750413,42.249587,-1.0,...,89897,https://today.yougov.com/topics/politics/track...,The Economist,0,0.010319,0.000900,0.964021,0.8,0.185695,2.281607e-05
42,YouGov,2025-05-23,2025-05-26,1486.0,rv,43.0,41.0,42.750413,41.249587,-2.0,...,90016,https://today.yougov.com/topics/politics/track...,The Economist,0,0.010077,0.001288,0.964021,0.8,0.185695,3.188028e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,Public Opinion Strategies,2025-02-25,2025-03-02,1000.0,rv,44.0,46.0,46.071409,43.928591,2.0,...,89658,https://paragoninstitute.org/medicaid/paragon-...,Paragon Health Institute,26,0.008267,0.000017,1.209225,0.8,1.000000,2.281622e-06
60,American Pulse Research & Polling,2025-06-23,2025-06-25,633.0,rv,46.6,42.4,46.600000,42.400000,-4.2,...,90198,https://drive.google.com/file/d/1fQWKf1QhBKMmx...,NaN,27,0.006577,0.005977,0.200000,0.8,1.000000,1.079006e-04
91,Ipsos,2025-08-22,2025-08-24,1022.0,a,38.0,34.0,37.612631,34.387369,-4.0,...,90508,https://www.ipsos.com/sites/default/files/Reut...,Reuters,28,0.008357,0.165432,0.711279,0.5,1.000000,8.435126e-03
96,Morning Consult,2025-08-29,2025-08-31,2202.0,rv,45.0,41.0,44.570341,41.429659,-4.0,...,90535,https://links.morningconsult.com/s/vb/zpOJTp1f...,NaN,29,0.012267,0.232560,0.200000,0.8,1.000000,7.830722e-03


In [43]:
approval_polls['end_date'].min()

'2024-12-11'

In [44]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 20), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['rep_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['dem_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['rep_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['dem_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Republicans':yes_averages, 'Democrats':no_averages}), pd.DataFrame({'Date':date_range, 'Republicans':yes_stdevs, 'Democrats':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [45]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Republicans,Democrats
0,2025-01-20,47.744515,44.665094
1,2025-01-21,47.744207,44.665024
2,2025-01-22,47.743900,44.664934
3,2025-01-23,47.743593,44.664822
4,2025-01-24,47.743288,44.664685
...,...,...,...
250,2025-09-27,44.163130,46.705372
251,2025-09-28,44.163219,46.705555
252,2025-09-29,43.459849,46.199823
253,2025-09-30,43.459767,46.200370


In [46]:
polls_df.shape

(106, 21)

In [47]:
display_df = polls_df[['pollster', 'start_date', 'end_date', 'sample_size', 'population', 'dem', 'rep', 'net',
                      'total_weights', 'url', 'sponsors']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'dem':'Democrats', 'rep':'Republicans', 'net':'Net', 'total_weights':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population'].map(lambda x: x.upper()).astype(str)
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Democrats', 'Republicans', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Democrats,Republicans,Net
0,"<a href=""https://www.scribd.com/document/92545...",2025-09-25,2025-09-29,1126 RV,1.055125e-01,44.0,40.0,-4.0
1,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-09-26,2025-09-29,1656 A,3.264894e-02,41.0,35.0,-6.0
2,"<a href=""https://www.nytimes.com/interactive/2...",2025-09-22,2025-09-27,1313 RV,1.934417e-01,47.0,45.0,-2.0
3,"<a href=""https://napolitannews.org/posts/gener...",2025-09-22,2025-09-24,2000 RV,9.294270e-02,45.0,46.0,1.0
4,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-09-17,2025-09-22,1000 LV,1.851776e-02,41.0,47.0,6.0
...,...,...,...,...,...,...,...,...
101,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,2.928032e-08,40.0,48.0,8.0
102,"<a href=""https://napolitannews.org/posts/gener...",2025-01-15,2025-01-16,1000 RV,5.078499e-08,44.0,51.0,7.0
103,"<a href=""https://www.cygn.al/wp-content/upload...",2025-01-09,2025-01-12,1500 LV,1.322723e-07,44.6,46.8,2.2
104,"<a href=""https://mclaughlinonline.com/pols/wp-...",2024-12-11,2024-12-16,1000 LV,3.218595e-09,42.0,47.0,5.0


In [48]:
display_df.to_csv('generic_ballot_polls_display_table.csv')

In [49]:
approval_avg['Net'] = approval_avg['Republicans'] - approval_avg['Democrats']
approval_avg

,Date,Republicans,Democrats,Net
0,2025-01-20,47.744515,44.665094,3.079422
1,2025-01-21,47.744207,44.665024,3.079183
2,2025-01-22,47.743900,44.664934,3.078966
3,2025-01-23,47.743593,44.664822,3.078772
4,2025-01-24,47.743288,44.664685,3.078603
...,...,...,...,...
250,2025-09-27,44.163130,46.705372,-2.542243
251,2025-09-28,44.163219,46.705555,-2.542336
252,2025-09-29,43.459849,46.199823,-2.739974
253,2025-09-30,43.459767,46.200370,-2.740603


In [50]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Republicans_x'] + 1.96*approval_trendline['Republicans_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Republicans_x'] - 1.96*approval_trendline['Republicans_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Democrats_x'] + 1.96*approval_trendline['Democrats_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Democrats_x'] - 1.96*approval_trendline['Democrats_y']
approval_trendline = approval_trendline.rename({
    'Republicans_x':'approve_mean',
    'Democrats_x':'disapprove_mean',
}, axis='columns').drop(['Republicans_y', 'Democrats_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-20,47.744515,44.665094,51.531964,43.957067,45.466159,43.864028
1,2025-01-21,47.744207,44.665024,51.532279,43.956135,45.466184,43.863864
2,2025-01-22,47.743900,44.664934,51.532737,43.955063,45.466216,43.863652
3,2025-01-23,47.743593,44.664822,51.533349,43.953838,45.466255,43.863388
4,2025-01-24,47.743288,44.664685,51.534129,43.952447,45.466303,43.863068
...,...,...,...,...,...,...,...
250,2025-09-27,44.163130,46.705372,49.376202,38.950057,52.715911,40.694834
251,2025-09-28,44.163219,46.705555,49.374921,38.951517,52.716462,40.694647
252,2025-09-29,43.459849,46.199823,49.596623,37.323075,52.376051,40.023595
253,2025-09-30,43.459767,46.200370,49.595205,37.324329,52.377367,40.023374


In [51]:
# remember: approve = Republicans, disapprove = Democrats - kept for DataWrapper setup simplicity
# Export generic ballot averages
approval_trendline.to_csv('generic_ballot_stats.csv')

In [52]:
# line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# # scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
# line.update_traces(hovertemplate=None)
# line.update_layout(hovermode='x unified')
# # scatter.update_traces(hovertemplate=None, hoverinfo='skip')
# fig_approve_CI = go.Figure([
#     go.Scatter(
#         name='Approve CI Upper Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
#         mode='lines',
#         marker=dict(color='#8be8f2'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),
#     go.Scatter(
#         name='Approve CI Lower Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
#         mode='lines',
#         marker=dict(color='#8be8f2'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),  
# ])
# fig_disapprove_CI = go.Figure([
#     go.Scatter(
#         name='Disapprove CI Upper Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
#         mode='lines',
#         marker=dict(color='#a3cbec'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),
#     go.Scatter(
#         name='Disapprove CI Lower Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
#         mode='lines',
#         marker=dict(color='#a3cbec'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),  
# ])

# fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
# fig.update_layout(
#     title='Congressional Approval Rating',
#     xaxis_title='Date',
#     yaxis_title='%',
#     legend_title='Legend',
#     hovermode='x unified'
# )
# fig.show()

In [53]:
approval_polls.head()

,pollster,start_date,end_date,sample_size,population,dem,rep,dem_adj,rep_adj,net,pred_plus_minus,poll_id,url,sponsors,pollster_id
9,YouGov,2025-02-16,2025-02-18,1603.0,a,35.0,37.0,34.750413,37.249587,2.0,0.047635,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
10,YouGov,2025-02-16,2025-02-18,1451.0,rv,42.0,41.0,41.750413,41.249587,-1.0,0.047635,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
23,YouGov,2025-04-05,2025-04-08,1741.0,a,39.0,36.0,38.750413,36.249587,-3.0,0.047635,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
24,YouGov,2025-04-05,2025-04-08,1558.0,rv,43.0,42.0,42.750413,42.249587,-1.0,0.047635,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
38,YouGov,2025-05-16,2025-05-19,1558.0,rv,43.0,42.0,42.750413,42.249587,-1.0,0.047635,89897,https://today.yougov.com/topics/politics/track...,The Economist,0


In [54]:
def net_col(df: pd.DataFrame, min_date: datetime.date):
#     df_copy = df.set_index(['Date'])
#     return df_copy['approve_mean'] - df_copy['disapprove_mean']
    date_range = pd.date_range(start=min_date, end=datetime.datetime.today(), freq='d', inclusive='both')
    data = df.copy()
    data['net_adj'] = data['rep_adj'] - data['dem_adj']
    net_averages = []
    net_stdevs = []
    issue_cols = {'Date': date_range, 'net':[], 'net_std':[]}
    for date in date_range:
    #         try:
            pipelined_df = avgs_pipeline(data, date)
    #         except TypeError:
    #             print('TypeError')
    #             print(date)
            net_avg = np.sum(pipelined_df['net_adj'] * pipelined_df['total_weights'])
            net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net_adj'] - net_avg)**2))

            net_averages.append(net_avg)
            net_stdevs.append(net_std)

    issue_cols['net'].extend(net_averages)
    issue_cols[f'net_std'].extend(net_stdevs)
        
    net_averages, net_stdevs = [], []
        
    return pd.DataFrame(issue_cols)

In [55]:
net_approval = net_col(approval_polls, datetime.date(2025, 1, 20))
net_approval

,Date,net,net_std
0,2025-01-20,3.079422,2.314717
1,2025-01-21,3.079183,2.315147
2,2025-01-22,3.078966,2.315675
3,2025-01-23,3.078772,2.316310
4,2025-01-24,3.078603,2.317062
...,...,...,...
250,2025-09-27,-2.542243,3.073552
251,2025-09-28,-2.542336,3.073906
252,2025-09-29,-2.739974,2.913742
253,2025-09-30,-2.740603,2.914490


In [56]:
net_approval['net_upper_CI'] = net_approval['net'] + 1.96*net_approval['net_std']
net_approval['net_lower_CI'] = net_approval['net'] - 1.96*net_approval['net_std']
net_approval

,Date,net,net_std,net_upper_CI,net_lower_CI
0,2025-01-20,3.079422,2.314717,7.616268,-1.457424
1,2025-01-21,3.079183,2.315147,7.616871,-1.458504
2,2025-01-22,3.078966,2.315675,7.617689,-1.459757
3,2025-01-23,3.078772,2.316310,7.618740,-1.461197
4,2025-01-24,3.078603,2.317062,7.620045,-1.462839
...,...,...,...,...,...
250,2025-09-27,-2.542243,3.073552,3.481919,-8.566405
251,2025-09-28,-2.542336,3.073906,3.482521,-8.567192
252,2025-09-29,-2.739974,2.913742,2.970961,-8.450908
253,2025-09-30,-2.740603,2.914490,2.971797,-8.453004


In [57]:
net_approval.to_csv('net_generic_ballot.csv')